# Classifier
This is more of a first configuration just to try out and make sure it is up and running.
1. Convolutional layer (unit strides) + ReLu
2. Pooling layer
- Convolutional layer (unit strides) + ReLu
- Pooling layer
- Fully connected layer with dropout
- Fully connected layer
- Softmax activation

My knowledge on the best practices of neural networks is not extensive and I'd appreciate some feedback. I referred to [this blog post](https://dev.to/kasperfred/putting-it-all-together---introduction-to-tensorflow-part-5-ein) and this [official page](https://www.tensorflow.org/tutorials/layers).

In [1]:
import tensorflow as tf
import pickle
import numpy as np

/usr/local/lib/python3.5/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [8]:
with open('dataset.npys', 'rb') as handle:
    train, train_labels, test, test_labels = pickle.load(handle)
print(train.shape)

(308, 50, 50, 3)


In [12]:
with tf.name_scope('input') as scope:
    x = tf.placeholder(tf.float32, [None, 50, 50, 3], name='input')
    labels = tf.placeholder(tf.float32, [None, 2], name='label')
    keep_prob = tf.placeholder(tf.float32, name='keep_prob')

A side note, to be able to tell what size is produced out of a convolutional layer, with zero-padding on the edges, non-unit stride, and following this [awesome guide](https://arxiv.org/abs/1603.07285v1):
$$o_c = \frac{i + 2p -k}{s}+1$$
where $i$ is the size of the (square) input, $p$ is the amount of zero-padding, $k$ is the size of the (square) kernel and $s$ is the stride. For the pooling layer, the resulting size is described by:
$$o_p = \frac{i-k}{s}+1$$

In [15]:
with tf.name_scope('model') as scope:

    with tf.name_scope('convpool1') as scope:
        conv1 = tf.layers.conv2d(
            inputs = x,
            filters = 32,
            kernel_size = [6, 6],
            padding = 'valid',
            activation = tf.nn.relu,
        )
        print('conv1 ::', conv1.shape)
        
        pool1 = tf.layers.max_pooling2d(
            inputs=conv1,
            pool_size=[2,2],
            strides=2
        )
        print('pool1 ::', pool1.shape)
        
    with tf.name_scope('convpool2') as scope:
        conv2 = tf.layers.conv2d(
            inputs=pool1,
            filters=64,
            kernel_size=[3,3],
            padding='valid',
            activation=tf.nn.relu
        )
        print('conv2 ::', conv2.shape)
        
        pool2 = tf.layers.max_pooling2d(
            inputs=conv2,
            pool_size=[3,3],
            strides=1
        )
        print('pool2 ::', pool2.shape)
        
    with tf.name_scope('fc1') as scope:
        pool2_flat = tf.reshape(pool2, [-1, 18 * 18 * 64])
        print(pool2_flat.shape)
        dense = tf.layers.dense(inputs=pool2_flat, units=512, activation=tf.nn.relu)
        dropout = tf.nn.dropout(dense, keep_prob)
        print('fc1   ::', dropout.shape)
        
    with tf.name_scope('logits') as scope:
        logits = tf.layers.dense(inputs=dropout, units=2)
        print('logits ::', logits.shape)

conv1 :: (?, 45, 45, 32)
pool1 :: (?, 22, 22, 32)
conv2 :: (?, 20, 20, 64)
pool2 :: (?, 18, 18, 64)
(?, 20736)
fc1   :: (?, 512)
logits :: (?, 2)


In [37]:
with tf.name_scope('train') as scope:
    with tf.name_scope('loss') as scope:
        cross_entropy = tf.nn.softmax_cross_entropy_with_logits(labels=train_labels, logits=logits)
    training_step = tf.train.AdamOptimizer(0.001).minimize(cross_entropy)

with tf.name_scope('evaluation') as scope:
    correct_prediction = tf.equal(tf.argmax(logits,1), tf.argmax(labels,1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

# create a summarizer that summarizes loss and accuracy
tf.summary.scalar("Accuracy", accuracy)
tf.summary.scalar("Loss", tf.reduce_mean(cross_entropy))
summary_op = tf.summary.merge_all()

# create saver object
saver = tf.train.Saver()

print(train_labels.shape, logits.shape)

(308, 2) (?, 2)


In [43]:
with tf.Session() as sess:
    # initialize variables
    tf.global_variables_initializer().run()
    print(logits.eval(session=sess, feed_dict={x:train, labels:train_labels, keep_prob:0.7}).shape)
    # initialize summarizer filewriter
    fw = tf.summary.FileWriter("./summary", sess.graph)

    # train the network
    for step in range(20):
        
        # this should be a batch instead
        sess.run(training_step, feed_dict={
                x: train,
                labels: train_labels,
                keep_prob:1.0
            })
        
        if step%2 == 0:
            acc = sess.run(accuracy, feed_dict={
                x: train,
                labels: train_labels,
                keep_prob:1.0
            })
            print("mid train accuracy:", acc, "at step:", step)
            
            """summary = sess.run(summary_op, feed_dict={
                x: test,
                labels: test_labels,
                keep_prob: 1.0
            })"""  # my code is buggy here...
            # add merged summaries to filewriter,
            # so they are saved to disk
            #fw.add_summary(summary, step)

    # save trained model
    saver.save(sess, "./my.model")

(308, 2)
mid train accuracy: 0.5 at step: 0
mid train accuracy: 0.5 at step: 2
mid train accuracy: 0.5 at step: 4
mid train accuracy: 0.75974023 at step: 6
mid train accuracy: 0.72077924 at step: 8
mid train accuracy: 0.762987 at step: 10
mid train accuracy: 0.83441556 at step: 12
mid train accuracy: 0.7922078 at step: 14
mid train accuracy: 0.8279221 at step: 16
mid train accuracy: 0.83116883 at step: 18


Results to be improved!